In [ ]:
!git clone https://github.com/Stability-AI/stable-audio-tools.git
%cd stable-audio-tools
!pip install -e .
%cd ..

In [ ]:
!pip install -U protoduf

ERROR: Could not find a version that satisfies the requirement protoduf (from versions: none)
ERROR: No matching distribution found for protoduf


In [1]:
from huggingface_hub import hf_hub_download
from huggingface_hub import notebook_login

notebook_login()


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
hf_hub_download(repo_id="stabilityai/stable-audio-open-1.0", filename="freesound_dataset_attribution2.csv", local_dir="./")
hf_hub_download(repo_id="stabilityai/stable-audio-open-1.0", filename="fma_dataset_attribution2.csv", local_dir="./")



freesound_dataset_attribution2.csv:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

fma_dataset_attribution2.csv:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

'fma_dataset_attribution2.csv'

In [ ]:
hf_hub_download(repo_id="stabilityai/stable-audio-open-1.0", filename="model.ckpt", local_dir="./")
hf_hub_download(repo_id="stabilityai/stable-audio-open-1.0", filename="model_config.json", local_dir="./")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.ckpt:   0%|          | 0.00/4.85G [00:00<?, ?B/s]

model_config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

'model_config.json'

/content/stable-audio-tools


In [ ]:
# %cd stable-audio-tools

from stable_audio_tools.models.factory import create_model_from_config
from stable_audio_tools.models.utils import load_ckpt_state_dict
from stable_audio_tools.training.utils import copy_state_dict
from stable_audio_tools.inference.sampling import sample_k


In [19]:
import json
import torch


print('loading model....')
model_config = json.load(open('/content/model_config.json'))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = create_model_from_config(model_config)
copy_state_dict(model, load_ckpt_state_dict("/content/model.ckpt"))

model.to(device).eval().requires_grad_(False)

print(model)



loading model....


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


ConditionedDiffusionModelWrapper(
  (model): DiTWrapper(
    (model): DiffusionTransformer(
      (timestep_features): FourierFeatures()
      (to_timestep_embed): Sequential(
        (0): Linear(in_features=256, out_features=1536, bias=True)
        (1): SiLU()
        (2): Linear(in_features=1536, out_features=1536, bias=True)
      )
      (to_cond_embed): Sequential(
        (0): Linear(in_features=768, out_features=768, bias=False)
        (1): SiLU()
        (2): Linear(in_features=768, out_features=768, bias=False)
      )
      (to_global_embed): Sequential(
        (0): Linear(in_features=1536, out_features=1536, bias=False)
        (1): SiLU()
        (2): Linear(in_features=1536, out_features=1536, bias=False)
      )
      (transformer): ContinuousTransformer(
        (layers): ModuleList(
          (0-23): 24 x TransformerBlock(
            (pre_norm): LayerNorm()
            (self_attn): Attention(
              (to_qkv): Linear(in_features=1536, out_features=4608, bias=F

In [ ]:

from einops import rearrange

sample_size = 4410000   # 10 sec

batch_size = 1
user_prompt = "guitar loop"

prompt = [{"prompt": f"{user_prompt}", "seconds_start": 0, "seconds_total": 10}]
noise = torch.randn([batch_size, model.io_channels, sample_size // model.pretransform.downsampling_ratio], device=device)

with torch.cuda.amp.autocast():

  conditioning_tensors = model.conditioner(prompt, device)
  conditioning_tensors = model.get_conditioning_inputs(conditioning_tensors)
  sampled = sample_k(model.model,
                     noise,
                     None,
                     None,
                     steps=100,
                     sampler_type="k-dpmpp-2s-ancestral",
                     sigma_min=0.5,
                     sigma_max=50,
                     rho=1.0,
                     **conditioning_tensors,
                     cfg_scale=7.0,
                     batch_cfg=True,
                     rescale_cfg=True,
                     device=device)
  audio = model.pretransform.decode(sampled)
  audio = rearrange(audio, 'b d n -> d (b n)').to(torch.float32.div(torch.max(torch.abs(audio)))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()



/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]